# Verodata kikkailut

In [1]:
import pandas as pd
import numpy as np
import requests
from json.decoder import JSONDecodeError

In [10]:
df = pd.read_csv('yhteiso_tuloverotus_julk_20{}.csv'.format(11), delimiter=';',encoding = "ISO-8859-1",
                decimal=',')
df.columns = df.columns.str.split('|', expand=True).droplevel(1).str.strip(' ')
#df = df[df['Verotettava tulo'] != 0]

for i in range(12, 19):
    df_t = pd.read_csv('yhteiso_tuloverotus_julk_20{}.csv'.format(i), delimiter=';',encoding = "ISO-8859-1",
                decimal=',')
    df_t.columns = df.columns
    #df_t = df_t[df_t['Verotettava tulo'] != 0]
    df = df.append(df_t)
    
#df = df[df.iloc[:,4:].abs().sum(1) != 0]
df.to_csv('verodata.csv')

In [3]:
df = pd.read_csv('verodata.csv', index_col=0)

C:\Users\lauri\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
records = pd.DataFrame()

In [4]:
i = 1500000
while i < 3000000:
    print('Trying: {}'.format(i))
    url = 'https://avoindata.prh.fi/tr/v1/publicnotices?totalResults=false&maxResults=1000' + \
        '&noticeRegistrationFrom=2011-01-01&noticeRegistrationTo=2018-12-31' + \
        '&resultsFrom={}'.format(i)
    r = requests.get(url)
    if r.status_code == 429:
        print('status 429')
        print('dumping to csv')
        records.to_csv('recs{}.csv'.format(i))
        break
    else:
        try:
            recs = r.json()['results']
            records = records.append(pd.DataFrame.from_records(recs).iloc[:,[0,1,2,3,5]])
            i += 1000
            if i % 100000 == 0:
                print('dumping to csv')
                records.to_csv('recs{}.csv'.format(i))
                records = pd.DataFrame()
        except JSONDecodeError:
            print('jsondecodeerror')
            continue

Trying: 1500000
status 429
dumping to csv


In [ ]:
import glob
import pandas as pd
import numpy as np
# get data file names

path='C:/Users/lauri/koodi/vero'

filenames = glob.glob(path + "/recs*.csv")
print(filenames)
dfs = []

for filename in filenames:
    
    dfs.append(pd.read_csv(filename, index_col=0))

records = pd.concat(dfs, ignore_index=True)

records.entryCodes = [eval(val) for val in records.entryCodes]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
records = records.join(pd.DataFrame(mlb.fit_transform(records.entryCodes), 
                                    columns=mlb.classes_, index=records.index))
records.set_index('recordNumber', inplace=True)

mask = records.iloc[:,4:].columns[records.iloc[:,4:].sum() < 100]
records['MUUMUU'] = records[mask].sum(1)
records = records.drop(mask, 1)
records.to_csv('records_processed.csv')

In [82]:
from bs4 import BeautifulSoup
import unicodedata
df = pd.DataFrame()
i = 1
for i in range(548):
    url = 'https://www.is.fi/yritykset/protestit/kaikki/?n={}'.format(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    table = soup.select('table[class^="companydata__table foldable companydata__table--protests"]')[0]
    links = [a['href'] for a in table.select('a')]
    sums = [unicodedata.normalize("NFKD", td.text).strip() for\
         td in table.select('td[class="sum"]')]
    dates = [unicodedata.normalize("NFKD", td.text).strip() for\
         td in table.select('td[class="date"]')]
    ids = []
    for url in links:
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
        table = soup.select('table[class^="companydata__table companydata__table--info"]')[0]
        ids.append(table.select('td')[1].text)
    df = df.append(pd.DataFrame([ids, sums, dates]).T)
    print('Page: ', i)
    print(df)

Page:  0
            0          1          2
0   2344323-4    2 005 €   6.3.2020
1   2997752-8    7 445 €   6.3.2020
2   2426674-3    7 460 €   6.3.2020
3   0804726-8   25 058 €   6.3.2020
4   2411194-4    6 106 €   6.3.2020
5   2914777-8   23 070 €   6.3.2020
6   2757332-7    2 889 €   6.3.2020
7   2848664-7    4 349 €   6.3.2020
8   2262056-9  511 496 €   6.3.2020
9   2756630-3      189 €  25.2.2020
10  2989042-2    2 303 €  25.2.2020
11  2128587-7    2 323 €  25.2.2020
12  2756687-2    2 366 €  25.2.2020
13  2640165-4    1 145 €  25.2.2020
14  2637093-1    2 306 €  25.2.2020
15  2589923-5    2 069 €  25.2.2020
16  1894802-7      284 €  25.2.2020
17  1480729-4      585 €  25.2.2020
18  2533491-2      448 €  25.2.2020
19  2495388-5      639 €  25.2.2020
20  2309202-4   14 426 €  25.2.2020
21  2150228-8    2 923 €  25.2.2020
22  2480360-9    1 282 €  25.2.2020
23  2338056-0      497 €  25.2.2020
24  2279065-6      601 €  25.2.2020
25  2296507-3    6 377 €  25.2.2020
26  2270568-5    8 

Page:  16
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2661274-2     590 €  20.2.2020
36  1703775-8   1 594 €  20.2.2020
37  0493917-0     910 €  20.2.2020
38  2117500-2     880 €  21.2.2020
39  2816239-4   1 797 €  21.2.2020

[680 rows x 3 columns]
Page:  17
            0          1          2
0   2344323-4    2 005 €   6.3.2020
1   2997752-8    7 445 €   6.3.2020
2   2426674-3    7 460 €   6.3.2020
3   0804726-8   25 058 €   6.3.2020
4   2411194-4    6 106 €   6.3.2020
..        ...        ...        ...
35  1722883-7  217 100 €  27.1.2020
36  1406922-8    1 896 €   3.3.2020
37  2069150-7    1 875 €   3.3.2020
38  2693922-7    6 222 €   3.3.2020
39  2839466-2    8 741 €   3.3.2020

[720 rows x 3 columns]
Page:  18
            0          1         2
0   2344323-4    2 005 €  6.3.2020


Page:  34
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2725678-6   1 924 €  22.1.2020
36  2342849-9   2 834 €  15.1.2020
37  2342849-9   3 007 €  15.1.2020
38  2360548-3   2 638 €   9.1.2020
39  2628668-4   3 442 €  16.1.2020

[1400 rows x 3 columns]
Page:  35
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  1097251-6   2 960 €  27.1.2020
36  1097251-6   2 998 €  27.1.2020
37  1097251-6   3 047 €  27.1.2020
38  1097251-6   3 099 €  27.1.2020
39  2758375-2   2 588 €   8.1.2020

[1440 rows x 3 columns]
Page:  36
            0          1          2
0   2344323-4    2 005 €   6.3.2020
1   2997

Page:  53
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  0952129-1   2 596 €  13.1.2020
36  0708230-1   5 225 €  23.1.2020
37  2656575-5   6 624 €  28.1.2020
38  2656575-5   3 798 €  28.1.2020
39  2656575-5   8 078 €  28.1.2020

[2160 rows x 3 columns]
Page:  54
            0         1         2
0   2344323-4   2 005 €  6.3.2020
1   2997752-8   7 445 €  6.3.2020
2   2426674-3   7 460 €  6.3.2020
3   0804726-8  25 058 €  6.3.2020
4   2411194-4   6 106 €  6.3.2020
..        ...       ...       ...
35  2774244-2   3 847 €  8.1.2020
36  2901753-5   3 835 €  8.1.2020
37  2358780-1  32 534 €  8.1.2020
38  2358780-1  17 307 €  8.1.2020
39  2252718-8  44 234 €  9.1.2020

[2200 rows x 3 columns]
Page:  55
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 

Page:  72
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2730729-9   7 962 €  13.1.2020
36  2730729-9   8 005 €  13.1.2020
37  2730729-9   2 056 €  13.1.2020
38  2730729-9   9 555 €  13.1.2020
39  2730729-9   9 506 €  13.1.2020

[2920 rows x 3 columns]
Page:  73
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2635028-7   2 291 €  31.1.2020
36  2635028-7   2 574 €  31.1.2020
37  2635028-7   2 245 €  31.1.2020
38  2635028-7   1 862 €  31.1.2020
39  2635028-7   2 184 €  31.1.2020

[2960 rows x 3 columns]
Page:  74
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   299775

...

Page:  469
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2492274-7   2 918 €  16.9.2019
36  2492274-7   2 299 €  16.9.2019
37  2492274-7   2 986 €  16.9.2019
38  2492274-7   1 886 €  16.9.2019
39  2492274-7   2 905 €  16.9.2019

[18800 rows x 3 columns]
Page:  470
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2278499-3   6 342 €  18.9.2019
36  2278499-3   6 428 €  18.9.2019
37  2278499-3   6 401 €  18.9.2019
38  2278499-3   6 796 €  18.9.2019
39  2278499-3   6 372 €  18.9.2019

[18840 rows x 3 columns]
Page:  471
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2

Page:  487
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2585062-6   4 097 €  11.9.2019
36  2585062-6   5 759 €  11.9.2019
37  2585062-6   2 895 €  11.9.2019
38  2585062-6   4 100 €  11.9.2019
39  2585062-6   2 890 €  11.9.2019

[19520 rows x 3 columns]
Page:  488
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2579893-6   1 734 €  30.9.2019
36  2579893-6   4 404 €  30.9.2019
37  2579893-6   4 338 €  30.9.2019
38  2579893-6   4 446 €  30.9.2019
39  2579893-6   1 748 €  30.9.2019

[19560 rows x 3 columns]
Page:  489
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2

Page:  505
            0         1           2
0   2344323-4   2 005 €    6.3.2020
1   2997752-8   7 445 €    6.3.2020
2   2426674-3   7 460 €    6.3.2020
3   0804726-8  25 058 €    6.3.2020
4   2411194-4   6 106 €    6.3.2020
..        ...       ...         ...
35  2473061-4   2 047 €   26.9.2019
36  2473061-4   2 821 €   26.9.2019
37  2473061-4  14 109 €   26.9.2019
38  2473061-4   2 761 €   26.9.2019
39  1518569-3   4 721 €  18.10.2019

[20240 rows x 3 columns]
Page:  506
            0          1           2
0   2344323-4    2 005 €    6.3.2020
1   2997752-8    7 445 €    6.3.2020
2   2426674-3    7 460 €    6.3.2020
3   0804726-8   25 058 €    6.3.2020
4   2411194-4    6 106 €    6.3.2020
..        ...        ...         ...
35  2438497-7   14 834 €   12.9.2019
36  2438497-7   14 900 €   12.9.2019
37  2438497-7   20 613 €   12.9.2019
38  1040631-1  503 848 €  17.10.2019
39  2790381-6   22 552 €  17.10.2019

[20280 rows x 3 columns]
Page:  507
            0         1           2
0  

Page:  523
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2319963-0  22 006 €  30.9.2019
36  1035262-4  27 687 €  30.9.2019
37  1795345-7  21 817 €  30.9.2019
38  1546716-4  24 720 €  30.9.2019
39  0991638-1  24 759 €  30.9.2019

[20960 rows x 3 columns]
Page:  524
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  1949899-6  20 958 €  30.9.2019
36  1840017-6  94 721 €  30.9.2019
37  2615002-3  15 795 €  30.9.2019
38  2264241-8  15 289 €  30.9.2019
39  1198933-6  58 153 €  30.9.2019

[21000 rows x 3 columns]
Page:  525
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2

Page:  541
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2648171-1  28 520 €  18.9.2019
36  2446097-6   6 992 €  17.9.2019
37  1037354-8   5 242 €  18.9.2019
38  2009695-1   1 735 €  18.9.2019
39  1890875-4   6 078 €  18.9.2019

[21680 rows x 3 columns]
Page:  542
            0         1          2
0   2344323-4   2 005 €   6.3.2020
1   2997752-8   7 445 €   6.3.2020
2   2426674-3   7 460 €   6.3.2020
3   0804726-8  25 058 €   6.3.2020
4   2411194-4   6 106 €   6.3.2020
..        ...       ...        ...
35  2437135-9  12 245 €  16.9.2019
36  2927515-5   9 780 €  16.9.2019
37  2663965-4     178 €   5.9.2019
38  1543907-5  60 000 €  16.9.2019
39  2599772-4   2 046 €  16.9.2019

[21720 rows x 3 columns]
Page:  543
            0         1         2
0   2344323-4   2 005 €  6.3.2020
1   299

In [84]:
df.reset_index(drop=True).to_csv('protestilista.csv')

In [76]:
from bs4 import BeautifulSoup
import unicodedata
df = pd.DataFrame()
i = 1
for i in range(548):
    url = 'https://www.is.fi/yritykset/protestit/kaikki/?n={}'.format(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    table = soup.select('table[class^="companydata__table foldable companydata__table--protests"]')[0]
    links = [a['href'] for a in table.select('a')]
    sums = [unicodedata.normalize("NFKD", td.text).strip() for\
         td in table.select('td[class="sum"]')]
    dates = [unicodedata.normalize("NFKD", td.text).strip() for\
         td in table.select('td[class="date"]')]
    ids = []
    for url in links:
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
        table = soup.select('table[class^="companydata__table companydata__table--info"]')[0]
        ids.append(table.select('td')[1].text)
    df = df.append(pd.DataFrame([ids, sums, dates]).T)
    print('Page: ', i)
    print(df)

In [2]:
from bs4 import BeautifulSoup
import unicodedata
i = 1000

In [3]:
df = pd.DataFrame()
for i in range(6015, 8000):
    url = 'http://www.protestilista.com/?page={}'.format(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    ids = [a['href'][7:] for a in soup.select('a[class=""]')]
    info = [a.text.strip() for a in soup.select('span[class="color-white-mute"]')]
    debtors = info[::3]
    sums = info[1::3]
    dates = info[2::3]
    df = df.append(pd.DataFrame([ids, debtors, sums, dates]).T)
    print('Got ', i)
    if i % 1000 == 0:
        df.to_csv('protestit2_{}.csv'.format(i))

Got  6015
Got  6016
Got  6017
Got  6018
Got  6019
Got  6020
Got  6021
Got  6022
Got  6023
Got  6024
Got  6025
Got  6026
Got  6027
Got  6028
Got  6029
Got  6030
Got  6031
Got  6032
Got  6033
Got  6034
Got  6035
Got  6036
Got  6037
Got  6038
Got  6039
Got  6040
Got  6041
Got  6042
Got  6043
Got  6044
Got  6045
Got  6046
Got  6047
Got  6048
Got  6049
Got  6050
Got  6051
Got  6052
Got  6053
Got  6054
Got  6055
Got  6056
Got  6057
Got  6058
Got  6059
Got  6060
Got  6061
Got  6062
Got  6063
Got  6064
Got  6065
Got  6066
Got  6067
Got  6068
Got  6069
Got  6070
Got  6071
Got  6072
Got  6073
Got  6074
Got  6075
Got  6076
Got  6077
Got  6078
Got  6079
Got  6080
Got  6081
Got  6082
Got  6083
Got  6084
Got  6085
Got  6086
Got  6087
Got  6088
Got  6089
Got  6090
Got  6091
Got  6092
Got  6093
Got  6094
Got  6095
Got  6096
Got  6097
Got  6098
Got  6099
Got  6100
Got  6101
Got  6102
Got  6103
Got  6104
Got  6105
Got  6106
Got  6107
Got  6108
Got  6109
Got  6110
Got  6111
Got  6112
Got  6113
Got  6114


KeyboardInterrupt: 

In [4]:
df.to_csv('protestit2_2.csv')